<a href="https://colab.research.google.com/github/akashahbahar/NLP-TFIDF-Multinomial-Naive-Bayes-Spam-Detection/blob/main/spam_detection_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np;
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#corpus = ["Topic sentences are similar to mini thesis statements.",
    #"Like a thesis statement, a topic sentence has a specific main point.",
    #"An essay is unified if all the paragraphs relate to the thesis."]
!pip install kaggle


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d ashfakyeafi/spam-email-classification

Dataset URL: https://www.kaggle.com/datasets/ashfakyeafi/spam-email-classification
License(s): apache-2.0
100% 207k/207k [00:00<00:00, 551kB/s]
100% 207k/207k [00:00<00:00, 550kB/s]


In [ ]:
!unzip spam-email-classification.zip

Archive:  spam-email-classification.zip
  inflating: email.csv               


In [ ]:
def preprocess_text(text):
  tokens = word_tokenize(text.lower())
  return [word for word in tokens if word.isalpha()]

In [ ]:
def compute_tf(doc):
  tf_dict = defaultdict(int)
  for word in doc:
    tf_dict[word] += 1
  doc_len = len(doc)
  tf_dict = {word: count / doc_len for word, count in tf_dict.items()}
  return tf_dict

In [ ]:
def compute_idf(corpus):
  idf_dict = defaultdict(int)
  total_docs = len(corpus)
  for doc in corpus:
    for word in set(doc):
      idf_dict[word] += 1
  idf_dict = {word: np.log(total_docs / (count+1)) for word, count in idf_dict.items()}
  return idf_dict

In [ ]:
def compute_tfidf(tf, idf):
  tfidf_dict = {word: tf_val * idf[word] for word, tf_val in tf.items()}
  return tfidf_dict

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('email.csv')

In [ ]:
df = pd.DataFrame(data)

In [ ]:
data_array = np.array(df)

In [ ]:
labels = data_array[:5572, 0].tolist() # POP() to remove last item
emails = data_array[:5572, 1].tolist()

In [ ]:
label_mapping = {'spam': 1, 'ham': 0}

email_class = [label_mapping[label] for label in labels]

In [ ]:
preprocessed_emails = [preprocess_text(email) for email in emails]

In [ ]:
emails_tf = [compute_tf(email) for email in preprocessed_emails]

In [ ]:
emails_idf = compute_idf(preprocessed_emails)

In [ ]:
emails_tfidf = [compute_tfidf(email_tf, emails_idf) for email_tf in emails_tf]

In [ ]:
tfidf_df = pd.DataFrame(emails_tfidf)
tfidf_df = tfidf_df.fillna(0)
tfidf_matrix = tfidf_df.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB()

In [ ]:
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Detailed classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.9587443946188341
              precision    recall  f1-score   support

         ham       0.97      0.99      0.98       966
        spam       0.89      0.79      0.84       149

    accuracy                           0.96      1115
   macro avg       0.93      0.89      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [ ]:
email1 = "Sorry, I'll call later in meeting."

In [ ]:
email2 = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [ ]:
user_input = email2

In [ ]:
preprocessed_input = [preprocess_text(input) for input in user_input]

In [ ]:
tf_input = [compute_tf(input) for input in preprocessed_input]

In [ ]:
tfidf_input = [compute_tfidf(tf, emails_idf) for tf in tf_input]

In [ ]:
# Ensure the feature vector matches the training vocabulary
vocabulary = list(emails_idf.keys())  # Or load it if saved separately
email_vector = np.array([tfidf_input[0].get(word, 0) for word in vocabulary]).reshape(1, -1)

In [ ]:
clf.predict(email_vector)

array(['spam'], dtype='<U4')

In [ ]:
prediction = clf.predict(email_vector)

# Interpret the result
if prediction[0] == 'spam':
    print("The email is spam.")
else:
    print("The email is not spam.")

The email is spam.
